# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
# Import Python packages 
import cassandra
import re
import os
import glob
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking our current working directory
print(os.getcwd())

# Getting our current folder and subfolder event data
filepath = os.getcwd() + '/event_data'
print(filepath)
# Creating a for loop to create a list of files and collecting each filepath
for root, dirs, files in os.walk(filepath):
    
# using glob to join the file path and roots with the subdirectories
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace
/home/workspace/event_data


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [ ]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# getting the total number of rows 
print(len(full_data_rows_list))

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''): #filtering out rows where artist name is blank
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

In [3]:
# checking number of rows in our csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Apache Cassandra coding

## Now using the CSV file titled event_datafile_new.csv, located within the Workspace directory. The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

## Initating Apache Cassandra 

#### Creating a Cluster

In [4]:
# Making a connection to a Cassandra instance 

from cassandra.cluster import Cluster
cluster = Cluster()

# Establishing a session to begin executing queries
session = cluster.connect()

#### Create Keyspace

In [5]:
# Creating a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)
except Exception as e:
    print(e)

#### Set Keyspace

In [6]:
# Connecting to our Keyspace
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Creating tables to run the following queries.

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

### Query 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

In [7]:
''' Creating table : Modeling table basis Query - Query uses sessionId and itemInSession to filter data
Hence sessionId is our Partition key (should enable evenly distribution of data) and itemInSession is our Clustering Key, together they make the 
Primary Key''' 

query = "CREATE TABLE IF NOT EXISTS song_by_session"
query = query + "(sessionId int, itemInSession int, artist text, song text, length float, PRIMARY KEY (sessionId, itemInSession))"
session.execute(query)

In [9]:
# Inserting data into our table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skipping header
    for line in csvreader:
## Assigning the INSERT statements into our `query` variable
        query = "INSERT INTO song_by_session (sessionId, itemInSession, artist, song, length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        ## Assigning column elements for each column in the INSERT statement.
        #session.execute(query, (line[0], line[9], float(line[5]), int(line[8]), int(line[3])))
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

In [10]:
## The SELECT statement for Query 1 (also verifies that the data was entered into the table correctly)
query = "SELECT artist, song, length FROM song_by_session WHERE sessionid = 338 AND iteminsession = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print (row.artist, row.song, row.length)
    #print (row)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### ---------- Query 2 --------------

In [11]:
## TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
''' Creating table : Modeling table basis Query - Query uses userid and sessionid to filter data, and itemInSession to sort the data
Hence userid is our Partition key (should enable evenly distribution of data) & sessionid and itemInSession (for ordering) is our Clustering Key. 
Together they make the Primary Key''' 

query = "CREATE TABLE IF NOT EXISTS artist_song_by_user"
query = query + "(userId int, sessionId int, itemInSession int, artist text, song text, firstName text, lastName text, \
                  PRIMARY KEY ((userId, sessionId), itemInSession))"
session.execute(query)                    

In [12]:
# Inserting data into our table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skipping header
    for line in csvreader:
## ## Assigning the INSERT statements into our `query` variable
        query = "INSERT INTO artist_song_by_user (userId, sessionId, itemInSession, artist, song, firstName, lastName)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        ## Assigning column elements for each column in the INSERT statement.
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))
        #session.execute(query, (line[0], line[9], line[1], line[4], int(line[10]), int(line[8]), int(line[3])))

In [13]:
## The SELECT statement for Query 2 (also verifies that the data was entered into the table correctly)

query = "SELECT artist, song, firstname, lastname FROM artist_song_by_user WHERE userid = 10 AND sessionid = 182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print (row.artist, row.song, row.firstname, row.lastname)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


### --------- Query 3 --------

In [15]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
''' Creating table : Modeling table basis Query - Query uses songs to filter data, but songs do not uniquely identify each row, so including userid.
Hence song is our Partition key and userid is our Clustering Key, together they make the Primary Key. We have kept user id as custering id because we 
want the data to be ordered in exactly the same way we will query for it, i.e. by user'''

query = "CREATE TABLE IF NOT EXISTS user_by_song"
query = query + "(song text, userid int, firstName text, lastName text, PRIMARY KEY (song, userid))"
session.execute(query)

In [16]:
# Inserting data into our table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skipping header
    for line in csvreader:
## Assigning the INSERT statements into our `query` variable
        query = "INSERT INTO user_by_song (song, userid, firstName, lastName)"
        query = query + "VALUES (%s, %s, %s, %s)"
        ## Assigning column elements for each column in the INSERT statement.
        #session.execute(query, (line[9], line[1], line[4], int(line[8])))
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

In [17]:
## The SELECT statement for Query 3 (also verifies that the data was entered into the table correctly)
query = "SELECT firstName, lastName FROM user_by_song WHERE song = 'All Hands Against His Own' "
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print (row.firstname, row.lastname)
    #print (row)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Droping the tables before closing out the sessions

In [18]:
query = "drop table song_by_session"
session.execute(query)
query = "drop table artist_song_by_user"
session.execute(query)
query = "drop table user_by_song"
session.execute(query)

### Closing the session and cluster connection¶

In [19]:
session.shutdown()
cluster.shutdown()